# Modeling of the Finnish 2019 election results data

This notebook is for training different machine learning algorithms on the data from the Finnish 2019 general elections. The output variable is the vote total per candidate and the input variables are different background features found in the data. (For more details, see the notebook about wrangling the said data.)

The main purpose of this exercise is to get some hands-on experience with the algorithms being tested on the data. In addition to that, I am naturally also curious to see if a machine learning model that predicts the vote totals reasonably well from the background variables can be found.

In [ ]:
# Importing the necessary libraries

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

Before any modeling can be done, it is of course necessary to load the data and split it into the training and testing samples. In the interests of making my results replicable, I am doing the splitting with a set random seed.

In [ ]:
# Reading the data from the .csv file into a dataframe
data = pd.read_csv('election_data.csv')

# Checking the data frame
data.head()

In [ ]:
# Splitting the data
train_data, test_data, train_votes, test_votes = train_test_split(
    data.iloc[:,:5], data.iloc[:, 5:], test_size = 0.2, random_state = 17)

# Checking the split data
for item in [train_data, test_data, train_votes, test_votes]:
    print(item.info())
    print(item.head(), '\n')